In [1]:
!pip install mysql-connector-python
!pip install pandas
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [3]:
connection = create_server_connection("localhost", "root", "")

MySQL Database connection successful


In [4]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [5]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [6]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [7]:
conn1 = create_db_connection("localhost", "root", "", "schedmaster")

MySQL Database connection successful


In [8]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [9]:
q1 = """
SELECT *
FROM students;
"""
results = read_query(conn1, q1)
for result in results:
  print(result)

('aaa111', 'John', 'Doe', 'CPSC', 120)


In [10]:
print(results[0][0])

aaa111


In [11]:
class Student(object):
    def __init__(self, id, firstName, lastName, major, requiredHours):
        self.id = id
        self.firstName = firstName
        self.lastName = lastName
        self.major = major
        self.requiredHours = requiredHours

In [12]:
q2 = """
SELECT *
FROM sections c;"""

In [13]:
sections = read_query(conn1, q2)

In [14]:
class CourseSect(object):
    def __init__(self, crn, courseId, sectNum, instructor, campus, location, startTime, endTime, startDate, endDate, days, cap, registered, attributes):
        self.crn = crn
        self.courseId = courseId
        self.sectNum = sectNum
        self.instructor = instructor
        self.campus = campus
        self.location = location
        self.startTime = startTime
        self.endTime = endTime
        self.startDate = startDate
        self.endDate = endDate
        self.days = days
        self.cap = cap
        self.registered = registered
        self.attributes = attributes
    def conflictsWith(self, course):
        if (self.startTime == None or course.startTime == None):
            return False
        else:
            timeOverlap = False
            dayOverlap = False
            if ((self.startTime < course.startTime and self.endTime > course.startTime) 
                or (self.startTime > course.startTime and self.startTime < course.endTime) 
                or self.startTime == course.startTime or self.endTime == course.endTime):
                timeOverlap = True
            for day in self.days:
                if (day in course.days): dayOverlap = True
            if (timeOverlap and dayOverlap): return True
            else: return False

In [15]:
allSections = []
for c in sections:
    newCourse = CourseSect(c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13],)
    allSections.append(newCourse)

In [16]:
for c in range(10):
    print(sections[c])

(20003, 'ESC1100', 0, 'Colleen Katherine Mikelson', 'D', 'XX INTERNET', None, None, datetime.date(2022, 1, 10), datetime.date(2022, 5, 3), 'N/A', 50, 47, 'Beyond the Classroom and Lower Division Credit and Nat. Sciences Non-Lab Courses and Potential Test Proctoring Fee')
(20004, 'ESC1500L', 0, 'Bradley Robert Reynolds', 'C', 'HOLT 225', datetime.timedelta(seconds=36000), datetime.timedelta(seconds=42600), datetime.date(2022, 1, 10), datetime.date(2022, 5, 3), 'M', 24, 24, 'Lab/Studio Fee and Lower Division Credit and Natural Science Lab Course')
(20005, 'ESC1500L', 1, 'Bradley Robert Reynolds', 'C', 'HOLT 225', datetime.timedelta(seconds=56400), datetime.timedelta(seconds=63000), datetime.date(2022, 1, 10), datetime.date(2022, 5, 3), 'M', 24, 23, 'Lab/Studio Fee and Lower Division Credit and Natural Science Lab Course')
(20006, 'ESC1500L', 2, 'Bradley Robert Reynolds', 'C', 'HOLT 225', datetime.timedelta(seconds=31800), datetime.timedelta(seconds=38400), datetime.date(2022, 1, 10), dat

In [17]:
print(allSections[0].startTime)

None


In [18]:
allSections[0].conflictsWith(allSections[1])

False

In [19]:
allSections[1].conflictsWith(allSections[0])

False

In [20]:
allSections[1].conflictsWith(allSections[6])

True

In [21]:
allSections[1].conflictsWith(allSections[2])

False

In [22]:
allSections[6].conflictsWith(allSections[1])

True

In [24]:
def nonconflicting(studCourses, allCourses):
    conf = []
    nonconf = []
    for course1 in studCourses:
        for course2 in allCourses:
            if (course1.conflictsWith(course2)):
                if course2 not in conf:
                    conf.append(course2)
    for course in allCourses:
        if course not in conf:
            nonconf.append(course)
    return nonconf

In [25]:
courseA = allSections[0]
courseB = allSections[2]
studCourses = [courseA, courseB]
testCourses = allSections[:10]
nonconf = nonconflicting(studCourses, testCourses)

In [26]:
for c in nonconf:
    print(str(c.startTime) + ', ' + str(c.endTime) + ', ' + c.days)

None, None, N/A
10:00:00, 11:50:00, M
8:50:00, 10:40:00, T
10:50:00, 12:40:00, T
13:40:00, 15:30:00, T
11:00:00, 11:50:00, MWF
13:00:00, 14:50:00, R
9:25:00, 10:40:00, TR
9:00:00, 9:50:00, MWF


In [27]:
for c in studCourses:
    print(str(c.startTime) + ', ' + str(c.endTime) + ', ' + c.days)

None, None, N/A
15:40:00, 17:30:00, M


In [28]:
from_db = []
for c in nonconf:
    x = (c.crn, c.courseId, c.sectNum, c.startTime, c.endTime, c.days)
    from_db.append(x)
columns = ["CRN", "ID", "Section #", "Start Time", "End Time", "Days"]
df = pd.DataFrame(from_db, columns=columns)

In [29]:
display(df)

,CRN,ID,Section #,Start Time,End Time,Days
0,20003,ESC1100,0,NaT,NaT,N/A
1,20004,ESC1500L,0,10:00:00,11:50:00,M
2,20006,ESC1500L,2,08:50:00,10:40:00,T
3,20007,ESC1500L,3,10:50:00,12:40:00,T
4,20008,ESC1500L,4,13:40:00,15:30:00,T
5,20009,ESC1510,0,11:00:00,11:50:00,MWF
6,20010,ESC1510L,0,13:00:00,14:50:00,R
7,20011,ESC4100,0,09:25:00,10:40:00,TR
8,20012,ESC4840,0,09:00:00,09:50:00,MWF
